In [47]:
"""

There is a correspondence between finite (T0) topological spaces and finite posets.
The goal is to represent finite top. spaces and also posets in python,
and to be able to convert between them. 

This correspondence is functorial, meaning that continuous functions between topological spaces
correspond to order-preserving functions between posets.

The correspondence is given via the 'specialization order' , which is defined as follows:
x <= y if and only if x is in the closure of the singleton {y}.
Conversely: given a poset, a topology is defined by the basic open sets
 U_x := {y | x <= y} 

Some functions I want to implement:
- Homology groups of a topological space
- Barycentric subdivision 
- Cartesian product of topological spaces
- Mapping space Hom(X,Y) of continuous functions between topological spaces
- Simplicial complexes
- Cone
- Suspension

An long-term goal is to use this for topological quantum error correction codes.
i.e. for each point in the space, there is a qubit, and the topology of the space
determines the stabilizer generators. 

"""
import itertools
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import simplicial as simp

class Top:
    ''' 

    '''
    
    def __init__(self, data):

        if type(data) == nx.DiGraph:
            # basis is dictionary of the basic open sets { key = x : value = {y | x <= y} }
            basis = {}
            for node in data.nodes:
                basis[node] = set(nx.descendants(data,node)).union({node})
            self.basis = basis
            self.points = set(data.nodes)

            

        elif type(data) == dict:
            self.basis = data
            self.points = set(data.keys())

    def cl(self,x):
        ''' 
        Returns the closure of a point x in the topological space: cl{x}
        '''
        s = set()
        for point in self.points:
            if x not in self.basis[point]:
                s = s.union(self.basis[point])
        return self.points.difference(s)
    
    def isleq(self,x,y):
        ''' 
        Returns True if x <= y, False otherwise
        with respect to the specialization order
        '''
        return x in self.cl(y)
    
    def ischain(self, subset):
        ''' 
        Given a subset of the topological space, returns True iff it is a chain
        (meaning each element is comparable to every other)
        '''
        chain = True
        for point in subset:
            for other in subset:
                chain = chain and (self.isleq(point,other) or self.isleq(other,point))
        return chain
    
    def all_chains(self):
        
        """ 
        
        Returns an array of the set of chains (x_0 <= x_1 <= \cdots <= x_n ) 
        w.r.t the specialization order of the topological space.
        
        """
        chains = []
        for n in range(1,len(self.points)+1):
            for chain in itertools.combinations(self.points,n):
                if self.ischain(chain):
                    chains.append(np.array(chain))
        return chains

def Top_to_graph(T : Top) -> nx.DiGraph:
    ''' 
    Given a topological space T, returns the poset corresponding to the specialization order.
    Here, the poset is represented as a directed graph via the networkx library.
    x<=y in the poset iff there exists an edge connecting between x and y in the graph
    
    '''
    G = nx.DiGraph()
    for x in T.points:
        for y in T.points:
            if T.isleq(x,y):
                G.add_edge(x,y)
    return G
                    
def Top_to_reduced_graph(T : Top) -> nx.DiGraph:
    ''' 
    Given a topological space T, returns the poset corresponding to the specialization order.
    Here, the poset is represented as a directed graph via the networkx library.
    x<=y in the poset iff there exists a directed path from x to y in the graph
    (including the length 0 path x->x).

    Currently, this will return an error if the topological space is not T_0,
    since in that case the specialization order is not a partial order, but a pre-order.
    Ideally, I would like to implement a function that returns the T_0 quotient of a topological space.
    '''
    G = nx.DiGraph()
    for node in T.points:
        G.add_node(node)
    for x in T.points:
        for y in T.points:
            if T.isleq(x,y) and x != y:
                G.add_edge(x,y)
    G = nx.transitive_reduction(G)
    
    return G




def Top_to_simp(T:Top) -> simp.SimplicialComplex:
    ''' 
    Given a topological space T, returns the simplicial complex corresponding to the specialization order.
    Here, the simplicial complex is represented as a list of simplices, where each simplex is a list of vertices.
    '''
    s = simp.SimplicialComplex()
    for n in range(len(T.all_chains)):
        s.addSimplex(fs = T.all_chains[n])
    return s

# G = nx.DiGraph()
# G.add_nodes_from([1,2,3,4])
# G.add_edges_from([(1,3),(1,4),(2,3),(2,4)])




T = Top({0:{0,1,2,3},1:{1,3},2:{2,3},3:{3}})
print( T.all_chains() )
print (Top_to_simp(T))
# Top_to_simp(T).draw()
# G = Top_to_graph(T)
# nx.draw(G,label=True,with_labels=True)
# plt.show

[array([0]), array([1]), array([2]), array([3]), array([0, 1]), array([0, 2]), array([0, 3]), array([1, 3]), array([2, 3]), array([0, 1, 3]), array([0, 2, 3])]


<>:86: SyntaxWarning: invalid escape sequence '\c'
<>:86: SyntaxWarning: invalid escape sequence '\c'
C:\Users\Connor\AppData\Local\Temp\ipykernel_25752\2811533314.py:86: SyntaxWarning: invalid escape sequence '\c'
  """
C:\Users\Connor\AppData\Local\Temp\ipykernel_25752\2811533314.py:86: SyntaxWarning: invalid escape sequence '\c'
  """


TypeError: object of type 'method' has no len()

In [27]:
"""
A class for continuous functions between topological spaces
"""
class Func: 

    def __init__(self, f, X : Top, Y : Top):
        self.graph = f
        self.domain = X
        self.codomain = Y
    """
    f is a dictionary of the form {x : f(x)}
    """
    
    def iscont(self):
        """
        Returns True if the function is continuous

        There are a couple things to check:
        1. f is actually a function, i.e.
          a. if x in X, then f.graph[x] is in Y.points
          b. if f.graph[x_1]=f.graph[x_2], then x_1 = x_2
        2. f is continuous with respect to the topologies of X and Y
          a. if x_1 <= x_2 in X, then f.graph[x_1] <= f.graph[x_2] in Y
        """
        if self.graph.keys() != self.domain.points:
            print ("That's not a function!" + " The domain is wrong!")
            return False 
        for x in self.domain.points:
            if self.graph[x] not in self.codomain.points:
                print ("That's not a function!" + " The codomain is wrong!")
                return False
        
        for x in self.domain.points:
            for y in self.domain.points:
                if self.domain.isleq(x,y) and not self.codomain.isleq(self.graph[x],self.graph[y]):
                    return False
        return True

# test a few examples
# S = Serpinski space
S = Top({0:{0,1},1:{1}})
# id should be continuous
id = Func({0:0,1:1},S,S)
print(id.iscont())
# example of something that isn't continuous
f = Func({0:1,1:0},S,S)
print(f.iscont())
# examples of a non-functions
g= Func({0:1,1:2},S,S)
h= Func({0:0},S,S)
print(g.iscont())
print(h.iscont())

True
False
That's not a function! The codomain is wrong!
False
That's not a function! The domain is wrong!
False


In [15]:
d = {0:0,1:2,2:3,3:5,4:7}
d1 = {0:d}
d2 ={0:d,1:d1}


{0: 2}